# Tyche Example with Simple PV Model

## Set up.

### One only needs to execute the following line once, in order to make sure recent enough packages are installed.

In [ ]:
#pip install numpy>=1.17.2 pandas>=0.25.1

### Import packages.

In [ ]:
import os
import sys
sys.path.insert(0, os.path.abspath("../../../src"))

In [ ]:
import numpy             as np
import matplotlib.pyplot as pl
import pandas            as pd
import re                as re
import scipy.stats       as st
import seaborn           as sb

# The `tyche` package is located at <https://github.com/NREL/portfolio/tree/master/production-function/framework/src/tyche/>.
import tyche             as ty

from copy import deepcopy

## Load data.

### The data are stored in a set of tab-separated value files in a folder.

In [ ]:
designs = ty.Designs(path = ".",
                     name = 'utility-pv.xlsx')

In [ ]:
investments = ty.Investments(path = ".",
                             name = 'utility-pv.xlsx')

### Compile the production and metric functions for each technology in the dataset.

In [ ]:
designs.compile()

## Examine the data.

### The `functions` table specifies where the Python code for each technology resides.

In [ ]:
designs.functions

Right now, only the style `numpy` is supported.

### The `indices` table defines the subscripts for variables.

In [ ]:
designs.indices

### The `designs` table contains the cost, input, efficiency, and price data for a scenario.

In [ ]:
designs.designs

### The `parameters` table contains additional techno-economic parameters for each technology.

In [ ]:
designs.parameters

### The `results` table specifies the units of measure for results of computations.

In [ ]:
designs.results

### The `tranches` table specifies multually exclusive possibilities for investments: only one `Tranch` may be selected for each `Cateogry`.

In [ ]:
investments.tranches

### The `investments` table bundles a consistent set of tranches (one per category) into an overall investment.

In [ ]:
investments.investments

## Evaluate the tranches in the dataset.

In [ ]:
tranche_results = investments.evaluate_tranches(designs, sample_count=1).metrics

In [ ]:
tranche_results.xs(1, level="Sample", drop_level=False)

### Save results.

In [ ]:
#tranche_results.to_csv("output/utility_pv/results.csv")

### Plot GHG Metric

In [ ]:
g = sb.boxplot(
    x="Tranche",
    y="Value",
    data=tranche_results.xs(
        "GHG",
        level="Index"
    ).reset_index()[["Tranche", "Value"]],
    order=[
        "Base PV"              ,
        "Slow Progress on PV"      ,
        "Moderate Progress on PV"  ,
        "Fast Progress on PV"      ,
    ]
)
g.set(ylabel="GHG [gCO2e / module]")
g.set_xticklabels(g.get_xticklabels(), rotation=30);

### Plot LCOE Metric

In [ ]:
g = sb.boxplot(
    x="Tranche",
    y="Value",
    data=tranche_results.xs(
        "LCOE",
        level="Index"
    ).reset_index()[["Tranche", "Value"]],
    order=[
        "Base PV"              ,
        "Slow Progress on PV"      ,
        "Moderate Progress on PV"  ,
        "Fast Progress on PV"      ,
    ]
)
g.set(ylabel="LCOE [USD / kWh]")
g.set_xticklabels(g.get_xticklabels(), rotation=30);

### Evaluate the investments in the dataset.

In [ ]:
investment_results = investments.evaluate_investments(designs, sample_count=50)

### Cost of investments

In [ ]:
investment_results.amounts

### Benefits of investments

In [ ]:
investment_results.metrics.xs(1, level="Sample", drop_level=False)

In [ ]:
investment_results.summary.xs(1, level="Sample", drop_level=False)

### Plot GHG metric

In [ ]:
g = sb.boxplot(
    x="Investment",
    y="Value",
    data=investment_results.metrics.xs(
        "GHG",
        level="Index"
    ).groupby(["Investment", "Sample"]).sum(numeric_only=True).reset_index()[["Investment", "Value"]],
    order=[
        "No R&D Spending",
        "Low R&D Spending"   ,
        "Medium R&D Spending",
        "High R&D Spending"  ,
    ]
)
g.set(ylabel="GHG[gCO2e/system]")
g.set_xticklabels(g.get_xticklabels(), rotation=15);

### Plot LCOE Metric

In [ ]:
g = sb.boxplot(
    x="Investment",
    y="Value",
    data=investment_results.metrics.xs(
        "LCOE",
        level="Index"
    ).groupby(["Investment", "Sample"]).sum(numeric_only=True).reset_index()[["Investment", "Value"]],
    order=[
        "No R&D Spending",
        "Low R&D Spending"   ,
        "Medium R&D Spending",
        "High R&D Spending"  ,
    ]
)
g.set(ylabel="LCOE[USD/kWh]")
g.set_xticklabels(g.get_xticklabels(), rotation=15);

## Multi-objective decision analysis

### Compute costs and metrics for tranches.

In [ ]:
tranche_results = investments.evaluate_tranches(designs, sample_count=50)

### Display the cost of each tranche.

In [ ]:
tranche_results.amounts

### Display the metrics for each tranche.

In [ ]:
tranche_results.summary

### Fit a response surface to the results.¶

The response surface interpolates between the discrete set of cases provided in the expert elicitation. This allows us to study funding levels intermediate between those scenarios.

In [ ]:
evaluator = ty.Evaluator(tranche_results)

Here are the categories of investment and the maximum amount that could be invested in each:

In [ ]:
evaluator.max_amount

In [ ]:
evaluator.units

### Example interpolation.
#### Let's evaluate the case where each category is invested in at half of its maximum amount.

In [ ]:
example_investments = evaluator.max_amount / 2
example_investments

In [ ]:
evaluator.evaluate(example_investments)

In [ ]:
evaluator.evaluate_statistic(example_investments, np.mean)

In [ ]:
evaluator.evaluate_statistic(example_investments, np.std)

In [ ]:
evaluator.evaluate_statistic(example_investments, lambda x: np.quantile(x, 0.1))

### ε-Constraint multiobjective optimization

### Example optimization.

#### Limit spending to $3M.

In [ ]:
investment_max = 3e6

In [ ]:
optimizer = ty.EpsilonConstraintOptimizer(evaluator)

### Require that the GHG reduction be at least 40 gCO2e/system and that the Labor wages not decrease.

In [ ]:
metric_min = pd.Series([40, 0], name = "Value", index = ["GHG", "Labor"])

### Compute the ε-constrained maximum for the LCOE.

In [ ]:
optimum = optimizer.opt_slsqp(
    "LCOE"                       ,
    sense = 'max',
    total_amount = investment_max,
    statistic    = np.mean       ,
)
optimum.exit_message

In [ ]:
np.round(optimum.amounts)

In [ ]:
optimum.metrics

In [ ]:
np.round(optimum.amounts)

In [ ]:
optimum.metrics